# Setup

In [11]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
# Standard library imports
from pydantic import BaseModel
from copy import deepcopy
import threading
import warnings
import queue
import json
import os

warnings.filterwarnings("ignore")
JWT_SECRET_API = !echo $(aws --profile "chat-prod_ro" secretsmanager get-secret-value --secret-id "arn:aws:secretsmanager:us-west-2:242659714806:secret:shared/cresta-server-jwt_secret-VDn5My" --query SecretString --output text) # type: ignore
os.environ["JWT_SECRET_API"] = json.loads(JWT_SECRET_API[0])["jwt-secret"]
os.environ["CONFIG_SERVICE_ADDR"] = "auth.chat-prod.internal.cresta.ai:443"
os.environ["CONFIG_USE_SECURE_CHANNEL"] = "true"

# Third party imports

from sentence_transformers import SentenceTransformer, util
from retry import retry

# Greyparrot imports
from greyparrot.llm.prompting import prompts as prompts_utils
from greyparrot.conversations.db import ConversationsDBConn
from greyparrot.multi_tenancy.v3_config import V3Config
from greyparrot.conversations.utils import get_chats
from greyparrot.chats_common import PartialChat
from greyparrot.common import get_logger


logger = get_logger(__name__)

# Local imports
from llm_proxy_client import LLMProxyDevClient

In [13]:
embedder = SentenceTransformer("all-mpnet-base-v2")

In [14]:
customer_id = "brinks"
profile_id = "care-voice"
usecase_id = "care-voice"
language_code = "en-US"

In [15]:
def get_chats_with_ids(chat_ids: list[str]) -> list[PartialChat]:
    customer_name: str = V3Config.short_name_from_ids(customer_id, profile_id)
    conv_db_conn: ConversationsDBConn = ConversationsDBConn.from_customer_name(customer_name)
    chats: list[PartialChat] = conv_db_conn.get_detailed_chats(
        customer_id=customer_id,
        profile_id=profile_id,
        usecase_id=usecase_id,
        language_code=language_code,
        conversation_ids=chat_ids,
        is_dev_user=False,
    )
    return chats

In [16]:
LLM_ENGINE: str = "gpt-4o-mini"
CONCURRENCY: int = 10

In [17]:
@retry(tries=1, delay=60, backoff=2, logger=logger)
def chat_completion(**kwargs):
    return LLMProxyDevClient("openai").beta.chat.completions.parse(**kwargs)

# Discovery

In [18]:
# TODO remove this after fixing speaker_role flips
flips: dict[str, str] = {"agent": "visitor", "visitor": "agent"}

In [19]:
def chat_to_prompt_text(chat: PartialChat) -> str:
    return "\n".join([
        f"{prompts_utils.speaker_role_str_for_prompts(flips[m.speaker_role.value]).capitalize()}: {m.text}"
        for m in chat.messages
    ])

In [21]:
SYSTEM_PROMPT_AGENT_WORKFLOW_DISCOVERY = """### Context and data description
You are a conversation analyst working for Brinks Home Security Call Center.

You will be given 1 conversation at a time. Each conversation is between a Brinks Call Center Agent and a Customer. Your primary goal is to extract workflows of steps which the Agent takes in **the given conversation** to help resolve the Customer's needs related to their home security system and services.

For each workflow, return:
- Product: Must be one of the following official Brinks products: (see below for some example products)

  PANELS & KEYPADS:
  * Security Panel (GE model)
  * Security Panel (Honeywell VISTA)
  * VISTA Security Panel
  * GoControl Security Panel
  * GoControl Panel
  * IQ4 Keypad
  * Secondary Keypad
  * Security Keypad
  * Smartbox

  CAMERAS:
  * Security Camera (ADCV723)
  * Front Door Camera
  * SkyBell Doorbell Camera
  * Doorbell Camera
  * Garage Camera
  * Patio Camera
  * Indoor Camera

  SENSORS:
  * Glass Break Sensor
  * Glass Break Detector
  * Garage Door Sensor
  * Window Sensor
  * Front Door Sensor
  * Motion Detector
  * Door/Window Sensor
  * Dining Room Window Sensor
  * Honeywell Smoke Detector
  * Living Room Smoke Detector
  * SRF433 Carbon Monoxide Monitor
  * Carbon Monoxide Detector
  * Fire Sensor
  * Hardwired Sensors
  * Honeywell Wireless Contact Sensor

  DIGITAL PRODUCTS:
  * Alarm.com Account
  * Mobile App
  * Smart Lock

  SERVICES:
  * Alarm Monitoring Service
  * Installation Service
  * Battery Replacement Service
  * Tech Support Services
  * Virtual Technician Appointment
  * Service Appointment
  * Field Service Appointment
  * Brinks HomePlus Plan
  * Brinks Home Plus Warranty

  ACCOUNT & BILLING:
  * Account Management
  * Billing Service
  * Home Security Account
  * Brinks Account Services
  * Brinks Home Security Account

  OTHER EQUIPMENT:
  * Transformer
  * Yard Sign
  * Alarm Permit

- Issue: Specific customer problem (e.g., False Alarms, Device Offline, Camera Not Recording, App Login Issues, Billing Questions, Account Changes)
- Steps: Detailed troubleshooting or resolution steps the Agent follows

Make sure to only use the exact product names from the list above, not general categories or variations.
Make sure the issue clearly describes the exact problem the customer is experiencing.
Make sure the steps are detailed and follow Brinks' standard operating procedures.
Make sure to only return the agent troubleshooting steps, not the customer's requests or other information.

Common scenarios include:
- Security system troubleshooting (panel issues, sensor malfunctions, connectivity problems)
- Camera and video recording issues
- Account management (billing, autopay, contact updates)
- Mobile app and portal assistance
- Service changes (moving, upgrading, cancellation)
- Installation and maintenance appointments

**Important**: There could be more than 1 workflow in a single conversation. There could also be no workflows in a single conversation. The workflows will be used to create troubleshooting guides to address similar customer needs in the future."""

In [22]:
class Flow(BaseModel):
    product: str
    issue: str
    steps: list[str]

class Flows(BaseModel):
    flows: list[Flow]

In [23]:
def discover_flow_in_chat(chat: PartialChat,
                          llm_engine: str = LLM_ENGINE) -> list[Flow]:
    logger.info(f"Discovering Agent flow in chat {chat.chat_name}")
    messages: list[dict[str, str]] = [
        {
            "role": "system",
            "content": SYSTEM_PROMPT_AGENT_WORKFLOW_DISCOVERY
        },
        {
            "role": "user",
            "content": chat_to_prompt_text(chat)
        },
    ]

    return chat_completion(model=llm_engine,
                            messages=messages,
                            response_format=Flows).choices[0].message.parsed.flows

In [24]:
def extract_flows_from_chats(chats: list[PartialChat],
                                concurrency: int=10) -> dict[str, list[Flow]]:
    lock: threading.Lock = threading.Lock()
    indexes: queue.Queue = queue.Queue()

    workflows: dict[str, list[Flow]] = {}
    for idx in range(len(chats)):
        indexes.put(idx)

    def workflow_labeler_worker():
        while True:
            try:
                idx = indexes.get(block=False)
            except queue.Empty:
                return
            chat = chats[idx]
            try:
                extracted_workflows: list[Flow] = discover_flow_in_chat(chat)
                with lock:
                    workflows[str(chat)] = extracted_workflows
                    if len(workflows) % 10 == 0:
                        print(f"Workflows from {len(workflows)} chats extracted!")
            except Exception as e:
                logger.warning(e, str(chat))
            indexes.task_done()

    logger.info(
        f"Starting processing {len(chats)} chats with {concurrency} workers")
    workers = [
        threading.Thread(target=workflow_labeler_worker)
        for _ in range(concurrency)
    ]
    # Start all workers
    for worker in workers:
        worker.start()
    for worker in workers:
        worker.join()
    logger.info(f"Finished processing all {len(chats)} chats")

    return workflows

In [25]:
test_chat: PartialChat = get_chats_with_ids(["0843c54c-6487-45ce-946a-cc6257484f54"])[0]
workflows: list[Flow] = discover_flow_in_chat(test_chat)

{"message": "use case id not set, fallback to use profile id care-voice", "name": "greyparrot.common.customers", "asctime": "2025-02-14 16:24:53", "levelname": "WARNING", "filename": "customers.py", "funcName": "__init__", "threadName": "MainThread", "status": "WARNING"}
{"message": "use case id not set, fallback to use profile id care-voice", "name": "greyparrot.common.customers", "asctime": "2025-02-14 16:24:55", "levelname": "WARNING", "filename": "customers.py", "funcName": "__init__", "threadName": "MainThread", "status": "WARNING"}


cmd: cresta-cli connstring -i voice-prod voice-prod brinks-care-voice-FKrGHU -r
cmd: cresta-cli connstring -i voice-prod voice-prod brinks-care-voice -r


{"message": "Discovering Agent flow in chat 0843c54c-6487-45ce-946a-cc6257484f54", "name": "__main__", "asctime": "2025-02-14 16:24:58", "levelname": "INFO", "filename": "<ipython-input-23-ea4193a4eca2>", "funcName": "discover_flow_in_chat", "threadName": "MainThread", "status": "INFO"}


In [26]:
print(chat_to_prompt_text(test_chat))

Customer: Technical support
Agent: Thank you for being the best part of Brinks Home. My name is Juan. Who do I have the pleasure of speaking with?
Customer: Debbie. Deborah Terryya.
Agent: Good morning, missus Deborah. How can I assist you?
Customer: Okay. On December 16th, I had Brinks installed. I had one. Camera put on my front door, and I had one put on the tireport, and then I went out of town for a couple weeks. And only my carport has been working since December 6th eight. And I clicked on the troubleshoot for my front door, and it just keeps coming up that there's a problem. I mean, I did I did everything it told me to do. I unplugged the power cord, and told me to wait 5 minutes. And it was trying to find what the problem was. And it is still not responding my front door.
Agent: The front door. The other
Customer: Which would be by my court
Agent: Okay. Gotcha. Let me get into the account real quick. And let me just double check what is happening to that sensor.
Customer: Yeah

In [27]:
print(f"Found {len(workflows)} workflows\n")
for i, flow in enumerate(workflows):
    print(f"[Agent Workflow #{i + 1}]")
    print(f"Product: {flow.product}")
    print(f"Issue: {flow.issue}")
    print("Steps:", end="\n* ")
    print("\n* ".join(flow.steps))
    if i < len(workflows) - 1:
        print("-" * 100)

Found 1 workflows

[Agent Workflow #1]
Product: Front Door Camera
Issue: Camera Not Recording
Steps:
* Verify customer details and account information.
* Ask customer to confirm the status of the front door camera.
* Instruct customer to check the LED status of the front door camera.
* Instruct customer to unplug the power supply for the front door camera and then wait for 30-60 seconds.
* Guide the customer to reconnect the power supply to the front door camera.
* Ask customer to check the LED status again after reconnection.
* If there is no LED light, ask the customer to test the power outlet with another device to ensure it is working properly.
* Confirm outlet is functional if other device charges successfully.
* Determine the power supply might be defective due to lack of response from the camera.
* Inform the customer of the need for a technician service appointment to fix the camera issue.


# 200 Recent Chats

In [28]:
data_set = get_chats(
    'brinks-care-voice', 
    '2024-06-01',
    '2024-12-01',
    200,)

{"message": "Reading from cache_chats-6cdad05445414399627e4b4e59113f84.pkl", "name": "cache-utils", "asctime": "2025-02-14 16:25:05", "levelname": "INFO", "filename": "cache.py", "funcName": "wrapper", "threadName": "MainThread", "status": "INFO"}
{"message": "Loading dataset from cache_chats-6cdad05445414399627e4b4e59113f84.pkl", "name": "dataset-common", "asctime": "2025-02-14 16:25:05", "levelname": "INFO", "filename": "common.py", "funcName": "load", "threadName": "MainThread", "status": "INFO"}


{"message": "Loading dataset succeeded!", "name": "dataset-common", "asctime": "2025-02-14 16:25:05", "levelname": "INFO", "filename": "common.py", "funcName": "load", "threadName": "MainThread", "status": "INFO"}


In [29]:
sampled_chat_ids = list(set([
    chat.chat_name
    for chat in (item if not isinstance(item, tuple) else item[0] for item in data_set)
    if hasattr(chat, 'chat_name')
]))
print(len(sampled_chat_ids))

200


# Agent Workflow Discovery

In [30]:
sampled_chats = get_chats_with_ids(sampled_chat_ids)

{"message": "use case id not set, fallback to use profile id care-voice", "name": "greyparrot.common.customers", "asctime": "2025-02-14 16:25:09", "levelname": "WARNING", "filename": "customers.py", "funcName": "__init__", "threadName": "MainThread", "status": "WARNING"}
{"message": "use case id not set, fallback to use profile id care-voice", "name": "greyparrot.common.customers", "asctime": "2025-02-14 16:25:09", "levelname": "WARNING", "filename": "customers.py", "funcName": "__init__", "threadName": "MainThread", "status": "WARNING"}


cmd: cresta-cli connstring -i voice-prod voice-prod brinks-care-voice-FKrGHU -r
cmd: cresta-cli connstring -i voice-prod voice-prod brinks-care-voice -r


In [31]:
workflows = extract_flows_from_chats(sampled_chats)

{"message": "Starting processing 199 chats with 10 workers", "name": "__main__", "asctime": "2025-02-14 16:25:21", "levelname": "INFO", "filename": "<ipython-input-24-e12d352d6e64>", "funcName": "extract_flows_from_chats", "threadName": "MainThread", "status": "INFO"}
{"message": "Discovering Agent flow in chat 000c68d2-6c21-4ddf-ab75-fb32b2ce0fc8", "name": "__main__", "asctime": "2025-02-14 16:25:21", "levelname": "INFO", "filename": "<ipython-input-23-ea4193a4eca2>", "funcName": "discover_flow_in_chat", "threadName": "Thread-5 (workflow_labeler_worker)", "status": "INFO"}
{"message": "Discovering Agent flow in chat 0017ec90-12c7-44dc-932c-f10b38ce0b9a", "name": "__main__", "asctime": "2025-02-14 16:25:21", "levelname": "INFO", "filename": "<ipython-input-23-ea4193a4eca2>", "funcName": "discover_flow_in_chat", "threadName": "Thread-6 (workflow_labeler_worker)", "status": "INFO"}
{"message": "Discovering Agent flow in chat 0039e60e-247f-4a8d-80d1-18d380344dce", "name": "__main__", "asc

{"message": "Discovering Agent flow in chat 01086912-66f0-4cdc-b38b-9941fedaca8d", "name": "__main__", "asctime": "2025-02-14 16:25:22", "levelname": "INFO", "filename": "<ipython-input-23-ea4193a4eca2>", "funcName": "discover_flow_in_chat", "threadName": "Thread-8 (workflow_labeler_worker)", "status": "INFO"}
{"message": "Discovering Agent flow in chat 01124bf9-4f07-4bf1-a86d-30c4fc6278d6", "name": "__main__", "asctime": "2025-02-14 16:25:23", "levelname": "INFO", "filename": "<ipython-input-23-ea4193a4eca2>", "funcName": "discover_flow_in_chat", "threadName": "Thread-7 (workflow_labeler_worker)", "status": "INFO"}
{"message": "Discovering Agent flow in chat 01532d84-1f4b-407f-84aa-54a29374b496", "name": "__main__", "asctime": "2025-02-14 16:25:23", "levelname": "INFO", "filename": "<ipython-input-23-ea4193a4eca2>", "funcName": "discover_flow_in_chat", "threadName": "Thread-13 (workflow_labeler_worker)", "status": "INFO"}
{"message": "Discovering Agent flow in chat 0167a38e-2cbd-42ef-

Workflows from 10 chats extracted!


{"message": "Discovering Agent flow in chat 0257d209-a041-47cc-baa8-70a785504964", "name": "__main__", "asctime": "2025-02-14 16:25:25", "levelname": "INFO", "filename": "<ipython-input-23-ea4193a4eca2>", "funcName": "discover_flow_in_chat", "threadName": "Thread-10 (workflow_labeler_worker)", "status": "INFO"}
{"message": "Discovering Agent flow in chat 0297c7d2-a6a2-4559-944a-4727df5b9b6f", "name": "__main__", "asctime": "2025-02-14 16:25:25", "levelname": "INFO", "filename": "<ipython-input-23-ea4193a4eca2>", "funcName": "discover_flow_in_chat", "threadName": "Thread-13 (workflow_labeler_worker)", "status": "INFO"}
{"message": "Discovering Agent flow in chat 034f5b00-e216-401d-8bd1-67a6914ee4e5", "name": "__main__", "asctime": "2025-02-14 16:25:25", "levelname": "INFO", "filename": "<ipython-input-23-ea4193a4eca2>", "funcName": "discover_flow_in_chat", "threadName": "Thread-11 (workflow_labeler_worker)", "status": "INFO"}
{"message": "Discovering Agent flow in chat 037b2032-8cff-4f0

Workflows from 20 chats extracted!


...

{"message": "Discovering Agent flow in chat 1769dc8b-7c1d-43cc-ad42-055239339176", "name": "__main__", "asctime": "2025-02-14 16:26:04", "levelname": "INFO", "filename": "<ipython-input-23-ea4193a4eca2>", "funcName": "discover_flow_in_chat", "threadName": "Thread-14 (workflow_labeler_worker)", "status": "INFO"}
{"message": "Discovering Agent flow in chat 176bdf21-0a06-490f-ad78-d506636940cc", "name": "__main__", "asctime": "2025-02-14 16:26:05", "levelname": "INFO", "filename": "<ipython-input-23-ea4193a4eca2>", "funcName": "discover_flow_in_chat", "threadName": "Thread-11 (workflow_labeler_worker)", "status": "INFO"}
{"message": "Discovering Agent flow in chat 17819caa-4212-4fc3-bf14-26a2fdcdb719", "name": "__main__", "asctime": "2025-02-14 16:26:05", "levelname": "INFO", "filename": "<ipython-input-23-ea4193a4eca2>", "funcName": "discover_flow_in_chat", "threadName": "Thread-12 (workflow_labeler_worker)", "status": "INFO"}
{"message": "Discovering Agent flow in chat 17bf716e-010e-45c

Workflows from 180 chats extracted!


{"message": "Discovering Agent flow in chat 1913f1d4-0f13-4177-819a-b342ecf2c421", "name": "__main__", "asctime": "2025-02-14 16:26:07", "levelname": "INFO", "filename": "<ipython-input-23-ea4193a4eca2>", "funcName": "discover_flow_in_chat", "threadName": "Thread-10 (workflow_labeler_worker)", "status": "INFO"}
{"message": "Discovering Agent flow in chat 1919f981-b0dc-4584-ab49-364bbc943425", "name": "__main__", "asctime": "2025-02-14 16:26:07", "levelname": "INFO", "filename": "<ipython-input-23-ea4193a4eca2>", "funcName": "discover_flow_in_chat", "threadName": "Thread-12 (workflow_labeler_worker)", "status": "INFO"}
{"message": "Discovering Agent flow in chat 195f3467-6a11-47cb-a831-45ff25c30f92", "name": "__main__", "asctime": "2025-02-14 16:26:08", "levelname": "INFO", "filename": "<ipython-input-23-ea4193a4eca2>", "funcName": "discover_flow_in_chat", "threadName": "Thread-6 (workflow_labeler_worker)", "status": "INFO"}
{"message": "Discovering Agent flow in chat 19747617-8b80-44a3

Workflows from 190 chats extracted!


{"message": "Finished processing all 199 chats", "name": "__main__", "asctime": "2025-02-14 16:26:21", "levelname": "INFO", "filename": "<ipython-input-24-e12d352d6e64>", "funcName": "extract_flows_from_chats", "threadName": "MainThread", "status": "INFO"}


In [33]:
sum([len(w) for w in workflows.values()])

246

In [34]:
all_workflows = []
for w in workflows.values():
    all_workflows.extend(w)

# Grouping Workflows

In [40]:
def group_workflows_by_product(workflows: list[Flow]) -> dict[str, list[Flow]]:
    
    retval = {}
    for w in workflows:
        if w.product not in retval:
            retval[w.product] = []
        retval[w.product].append(w)

    print(f"Found {len(retval)} products in the workflows")
        
    return retval

In [41]:
def group_workflows_by_issues(workflows: list[Flow], embedder: SentenceTransformer, semantic_threshold: float) -> dict[tuple[str], list[Flow]]:
    
    issues = [w.issue for w in workflows]
    embeddings = embedder.encode(issues, convert_to_tensor=True)
        
    issue_groups, solo_issues = [], []
    for idx, _ in enumerate(issues):
        existing_groups = [group for group in issue_groups if idx in group]
        if existing_groups:
            assert len(
                existing_groups) == 1, "A workflow should only be in 1 group"
            continue

        scores = util.cos_sim(embeddings[idx:idx + 1],
                              embeddings).cpu().tolist()[0]
        matches = [
            i for i, score in enumerate(scores) if score > semantic_threshold
        ]
        filtered_matches = [i for i in matches if i != idx]
        if filtered_matches:
            outstanding_groups = []
            new_group = filtered_matches + [idx]
            for group in issue_groups:
                if set(filtered_matches).intersection(set(group)):
                    new_group.extend(group)
                else:
                    outstanding_groups.append(group)
            issue_groups = outstanding_groups + [list(set(new_group))]
        else:
            solo_issues.append(idx)

    print(f"Found {len(solo_issues)} # of Solo Issues.")
    print(f"Found {len(issue_groups)} # Groups of Issues.")

    # Create mapping of indices to product names
    idx_to_issue = {i: p for i, p in enumerate(issues)}
    
    retval = {}

    for group in issue_groups:
        group_key = tuple(set([idx_to_issue[idx] for idx in group]))
        retval[group_key] = [workflows[idx] for idx in group]

    for idx in solo_issues:
        retval[(workflows[idx].issue,)] = [workflows[idx]]

    return retval

In [42]:
tree = group_workflows_by_product(all_workflows)

Found 46 products in the workflows


In [43]:
for product in tree.keys():
    tree[product] = group_workflows_by_issues(tree[product], embedder, semantic_threshold=0.7)

Found 11 # of Solo Issues.
Found 2 # Groups of Issues.
Found 3 # of Solo Issues.
Found 0 # Groups of Issues.
Found 5 # of Solo Issues.
Found 2 # Groups of Issues.
Found 6 # of Solo Issues.
Found 1 # Groups of Issues.
Found 8 # of Solo Issues.
Found 1 # Groups of Issues.
Found 6 # of Solo Issues.
Found 2 # Groups of Issues.
Found 13 # of Solo Issues.
Found 2 # Groups of Issues.
Found 6 # of Solo Issues.
Found 1 # Groups of Issues.
Found 0 # of Solo Issues.
Found 1 # Groups of Issues.
Found 7 # of Solo Issues.
Found 0 # Groups of Issues.
Found 1 # of Solo Issues.
Found 0 # Groups of Issues.
Found 2 # of Solo Issues.
Found 1 # Groups of Issues.
Found 5 # of Solo Issues.
Found 4 # Groups of Issues.
Found 0 # of Solo Issues.
Found 1 # Groups of Issues.
Found 1 # of Solo Issues.
Found 0 # Groups of Issues.
Found 2 # of Solo Issues.
Found 0 # Groups of Issues.
Found 1 # of Solo Issues.
Found 1 # Groups of Issues.
Found 1 # of Solo Issues.
Found 0 # Groups of Issues.
Found 5 # of Solo Issues.


In [44]:
leafs = 0
for product in tree.keys():
    for issue in tree[product].keys():
        leafs += 1
print(f"Found {leafs} leafs in the tree")

Found 150 leafs in the tree


In [29]:
"""
{
    "camera": [camera_flow1, camera_flow2, camera_flow3],
    "alarm": [alarm_flow4, alarm_flow5, alarm_flow6],
    "TV": [tv_flow1]
}
"""

"""
{
    "camera": {
        "issue1": [camera_issue1_flow1, camera_issue1_flow2, camera_issue1_flow3],
        "issue2": [camera_issue2_flow1, camera_issue2_flow2, camera_issue2_flow3],
    },
    "alarm": {
        "issue1": [alarm_issue1_flow1, alarm_issue1_flow2, alarm_issue1_flow3],
        "issue2": [alarm_issue2_flow1, alarm_issue2_flow2, alarm_issue2_flow3],
    },
}
"""

"""
{
    "camera": {
        "issue1": troubleshooting_steps,
        "issue2": troubleshooting_steps,
    },
    "alarm": {
        "issue1": troubleshooting_steps,
        "issue2": troubleshooting_steps,
    },
}
"""

'\n{\n    "camera": {\n        "issue1": troubleshooting_steps,\n        "issue2": troubleshooting_steps,\n    },\n    "alarm": {\n        "issue1": troubleshooting_steps,\n        "issue2": troubleshooting_steps,\n    },\n}\n'

# Generate Troubleshooting Guide for each leaf in the tree

In [45]:
class TroubleshootingGuide(BaseModel):
    steps: list[str]

In [46]:
def concatenate_flows(flows: list[Flow]) -> str:
    conversations = []
    for i, flow in enumerate(flows):
        conversation = f"Conversation {i+1}:\n" + "\n".join(f"* {step}" for step in flow.steps)
        conversations.append(conversation)
    return "\n\n".join(conversations)


In [47]:
GENERATE_TROUBLESHOOTING_GUIDE_PROMPT = """\
You are a conversation analyst working for Brinks Home Security Call Center.

You will be given the troubleshooting steps taken by a call center agent of 1 or more conversations at a time. Your primary goal is to generate an overall troubleshooting guide for the given product and issue.

The troubleshooting guide must consider all scenarios and steps from the given conversations. Do not produce any steps that are not present in the given conversations. Only produce an overall troubleshooting guide after gaining information from all the conversations.

You must produce an ordered list of steps that the agent can follow to help resolve the customer's needs related to the product and issue.
"""

In [48]:
def generate_troubleshooting_guide(product: str, issue: str, llm_engine: str = LLM_ENGINE) -> str:
    messages: list[dict[str, str]] = [
        {
            "role": "system",
            "content": GENERATE_TROUBLESHOOTING_GUIDE_PROMPT
        },
        {
            "role": "user",
            "content": concatenate_flows(tree[product][issue])
        },
    ]

    return chat_completion(model=llm_engine,
                            messages=messages,
                            response_format=TroubleshootingGuide).choices[0].message.parsed.steps

In [49]:
def generate_troubleshooting_guides(tree: dict[str, dict[str, list[Flow]]], llm_engine: str = LLM_ENGINE, concurrency: int=10) -> None:
    lock: threading.Lock = threading.Lock()
    indexes: queue.Queue = queue.Queue()
    
    # Create flat list of (product, issue) pairs to process
    tasks = []
    for product in tree.keys():
        for issue in tree[product].keys():
            tasks.append((product, issue))
            
    for idx in range(len(tasks)):
        indexes.put(idx)

    def troubleshooting_guide_worker():
        while True:
            try:
                idx = indexes.get(block=False)
            except queue.Empty:
                return
                
            product, issue = tasks[idx]
            try:
                guide = generate_troubleshooting_guide(product, issue, llm_engine)
                with lock:
                    tree[product][issue] = guide
                if idx % 10 == 0:
                    print(f"Generated guides for {idx} product/issue pairs")
            except Exception as e:
                logger.warning(f"Error generating guide for {product}/{issue}: {e}")
            indexes.task_done()

    logger.info(f"Starting guide generation for {len(tasks)} product/issue pairs")
    workers = [
        threading.Thread(target=troubleshooting_guide_worker)
        for _ in range(concurrency)
    ]
    for worker in workers:
        worker.start()
    for worker in workers:
        worker.join()
    logger.info("Finished generating all guides")
    print("Finished generating all guides")

In [50]:
troubleshooting_guide = deepcopy(tree)

In [51]:
generate_troubleshooting_guides(troubleshooting_guide)

{"message": "Starting guide generation for 150 product/issue pairs", "name": "__main__", "asctime": "2025-02-14 16:35:07", "levelname": "INFO", "filename": "<ipython-input-49-f882fd4f0c39>", "funcName": "generate_troubleshooting_guides", "threadName": "MainThread", "status": "INFO"}


Generated guides for 10 product/issue pairs
Generated guides for 0 product/issue pairs
Generated guides for 20 product/issue pairs
Generated guides for 30 product/issue pairs
Generated guides for 50 product/issue pairs
Generated guides for 40 product/issue pairs
Generated guides for 60 product/issue pairs
Generated guides for 70 product/issue pairs
Generated guides for 80 product/issue pairs
Generated guides for 110 product/issue pairs
Generated guides for 100 product/issue pairs
Generated guides for 90 product/issue pairs
Generated guides for 130 product/issue pairs
Generated guides for 120 product/issue pairs
Generated guides for 140 product/issue pairs


{"message": "Finished generating all guides", "name": "__main__", "asctime": "2025-02-14 16:37:11", "levelname": "INFO", "filename": "<ipython-input-49-f882fd4f0c39>", "funcName": "generate_troubleshooting_guides", "threadName": "MainThread", "status": "INFO"}


Finished generating all guides


In [52]:
for product in troubleshooting_guide.keys():
    print(f"Product: {product}")
    print(f"{'-'*100}\n{'-'*100}")
    for issue in troubleshooting_guide[product].keys():
        print(f"**Issue: {issue}**")
        print("\n".join(troubleshooting_guide[product][issue]))
        print("-"*100)
    print("-"*100)

Product: Brinks Home Security Account
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
**Issue: ('Account Changes and Access Permissions', 'Account Changes')**
Verify the customer's identity using their phone number, address, and account code word.
Confirm if the customer is requesting cancellation or service-related issues.
If cancellation is requested, inform the customer that the cancellation process is handled by the loyalty department, provide the direct number (8443812351), and advise them to call during business hours.
If the customer wishes to change payment due dates, confirm their request and make the adjustment as necessary.
If the customer has recently moved, inform them that they need to contact the account ownership transfers department, provide the contact number (800-972-8956), and mention the department's hours (Monday

In [53]:
# Function to export troubleshooting guide to Markdown
def export_to_markdown(troubleshooting_guide, output_file="troubleshooting_guide.md"):
    with open(output_file, "w") as f:
        for product in troubleshooting_guide.keys():
            f.write(f"# Product: {product}\n")
            for issue in troubleshooting_guide[product].keys():
                # Write issue and steps
                f.write(f"## **Issue: {issue}**\n\n")
                f.write("\n".join([f"- {step}" for step in troubleshooting_guide[product][issue]]) + "\n")

# Example: Export the existing troubleshooting_guide
export_to_markdown(troubleshooting_guide)